In [1]:
import numpy as np

In [2]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("~/Downloads/frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin", binary=True, unicode_errors="ignore")

In [3]:
import pandas as pd

In [4]:
dico = pd.read_csv("/home/olojkine/Downloads/lexique-grammalecte-fr-v7.0.txt", skiprows=15, sep='\t')
dico.head(100)

,id,fid,Flexion,Lemme,Étiquettes,Métagraphe (β),Metaphone2,Notes,Sémantique,Étymologie,Sous-dictionnaire,Google 1-grams,Wikipédia,Wikisource,Littérature,Total occurrences,Doublons,Multiples,Fréquence,Indice de fréquence
0,139424,1,de,de,mg det epi inv,de,T,NaN,@,NaN,*,8818296701,94780970,37051349,6972501,4478550761,1,1,3.037618,9
1,232453,1,de,de,mg prep prepv,de,T,NaN,@,NaN,*,8818296701,94780970,37051349,6972501,4478550761,1,1,3.037618,9
2,145241,1,et,et,mg cjco,é,AT,NaN,@,NaN,*,3867624351,22249966,13925141,3823323,3907622781,1,0,2.650381,9
3,180587,1,à,à,mg prep prepv,a,A,NaN,@,NaN,*,3221974801,20826283,11299713,2919484,3257020281,1,0,2.209104,9
4,214633,1,des,des,mg det epi pl,des,TS,NaN,@,NaN,*,2764279382,16094864,7910581,1846193,2790131020,1,0,1.892432,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,140056,1,dire,dire,v3__tnq__a infi,dir,TR,NaN,NaN,NaN,*,119864550,118759,437031,116722,119973532,1,1,0.081373,8
96,125092,1,Paris,Paris,npr epi inv,paris,PRS,NaN,cité,NaN,*,117611421,1502567,324787,48218,119486993,1,0,0.081043,8
97,134561,1,celui,celui,mg prodem mas sg,Selui,SL,NaN,@,lat,*,117216049,247581,352349,80429,117896408,1,0,0.079964,8
98,162864,1,pays,pays,nom mas inv,pais,PS,NaN,NaN,NaN,*,115805702,1057472,308409,83306,117254889,1,0,0.079529,8


In [5]:
import requests
def req_word(word):
    r = requests.post("https://cemantix.herokuapp.com/score", data={"word": word}).json()
    return r.get("score", np.nan)

In [6]:
req_word("faire")

0.0823

In [9]:
import time
import random
vocab = set(dico[(dico.Fréquence > 1e-5) & (dico.Étiquettes.str.match("nom|v[123]|adj"))].Lemme)
vocab.intersection_update(model.index_to_key)
i = 0
while len(vocab) > 0:
    i += 1
    df = dico[dico.Lemme.isin(vocab) & (dico.Fréquence < 0.05)]
    word = df.loc[df.Fréquence.idxmax()].Lemme
    print(f"{i: 2d}. Il me reste {len(vocab):04} mots à essayer. J'essaie “{word}”. ", end="")
    score = req_word(word)
    vocab.remove(word)
    time.sleep(.1)
    if np.isnan(score):
        print(" Cémantix ne connaît pas ce mot.")
        continue
    if score == 1:
        print(" GAGNÉ")
        break
    print(f"Score: {100*score:.2f}.")
    tops = model.most_similar(word, topn=250_000)
    dmin = 0.2
    if tops[-1][1] < score - dmin:
        vocab.intersection_update({
            word
            for word, dist in tops
            if abs(dist-score) < dmin
        })

 1. Il me reste 25078 mots à essayer. J'essaie “avoir”. Score: 11.00.
 2. Il me reste 21716 mots à essayer. J'essaie “état”. Score: 3.52.
 3. Il me reste 21434 mots à essayer. J'essaie “corps”. Score: -0.24.
 4. Il me reste 20273 mots à essayer. J'essaie “n”.  Cémantix ne connaît pas ce mot.
 5. Il me reste 20272 mots à essayer. J'essaie “nom”. Score: -2.23.
 6. Il me reste 19633 mots à essayer. J'essaie “question”. Score: 4.41.
 7. Il me reste 19190 mots à essayer. J'essaie “guerre”. Score: 8.20.
 8. Il me reste 18466 mots à essayer. J'essaie “voir”. Score: 1.45.
 9. Il me reste 18205 mots à essayer. J'essaie “contre”. Score: 22.72.
 10. Il me reste 6250 mots à essayer. J'essaie “système”. Score: -3.57.
 11. Il me reste 5927 mots à essayer. J'essaie “roi”. Score: 2.23.
 12. Il me reste 5674 mots à essayer. J'essaie “suite”. Score: 10.26.
 13. Il me reste 5227 mots à essayer. J'essaie “prix”. Score: 6.64.
 14. Il me reste 5146 mots à essayer. J'essaie “français”. Score: 7.31.
 15. Il m